In [1]:
import os
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from keras import metrics
from keras.optimizers import Adam
from time import time
from IPython.display import display

Using TensorFlow backend.


In [2]:
experiment_name='mlhw-mlp-no-preprocess'

## Load Dataset

In [3]:
data_val = pd.read_csv('dataset/train_values.csv')
data_labels = pd.read_csv('dataset/train_labels.csv')

# Merge values and label
data = data_val.merge(data_labels, on='patient_id')
data = data.drop(columns='patient_id')

# Convert string values to integers
data["thal"] = data["thal"].astype('category')
data["thal"] = data["thal"].cat.codes

# Split dataset
train = data.sample(frac=0.8)
validate = data.sample(frac=0.2)

xsize = train.shape[1] - 1
ysize = 1
label_column = 'heart_disease_present'
cols = list(train.columns)
cols.remove(label_column)



In [4]:
display(train.head())

,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina,heart_disease_present
30,2,1,130,3,0,0,2,214,2.0,1,41,168,0,0
0,1,1,128,2,0,0,2,308,0.0,1,45,170,0,0
174,1,1,140,4,0,0,0,239,1.2,1,54,160,0,0
100,1,1,140,3,0,0,2,321,0.0,1,39,182,0,0
154,2,2,130,3,1,0,0,263,1.2,0,62,97,0,1


## Define Model 

In [7]:
model = Sequential()
model.add(BatchNormalization(input_shape=(xsize,)))
model.add(Activation('relu'))
model.add(Dense(256))

model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(128))

model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(64))

model.add(Dense(ysize))
model.add(Activation('sigmoid'))

opt = keras.optimizers.rmsprop(lr=initial_lr, decay=1e-6)

model.compile(
    loss='binary_crossentropy',
    optimizer=rmsprop(lr=0.001),
    metrics=[metrics.binary_accuracy]
)

## Training

In [8]:
epochs = 100
batch_size = 32

model.fit(
    train[cols].values,
    train[label_column].values,
    batch_size=batch_size,
    epochs=epochs,
    shuffle=True,
    verbose=1,
    validation_data=(validate[cols].values,validate[label_column].values)
)

# Save model and weights
save_dir=os.path.join(os.getcwd(), 'saved_models')
model_name='keras_mlwh_dnn_{}.h5'.format(time())
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

[  2.   1. 130.   3.   0.   0.   2. 214.   2.   1.  41. 168.   0.]
Train on 144 samples, validate on 36 samples
Epoch 1/100
144/144 [==============================] - 2s 11ms/step - loss: 0.5873 - binary_accuracy: 0.7014 - val_loss: 0.3800 - val_binary_accuracy: 0.8889
Epoch 2/100
144/144 [==============================] - 0s 301us/step - loss: 0.4345 - binary_accuracy: 0.8194 - val_loss: 0.4735 - val_binary_accuracy: 0.8333
Epoch 3/100
144/144 [==============================] - 0s 339us/step - loss: 0.3200 - binary_accuracy: 0.8681 - val_loss: 0.4210 - val_binary_accuracy: 0.8333
Epoch 4/100
144/144 [==============================] - 0s 314us/step - loss: 0.2696 - binary_accuracy: 0.8889 - val_loss: 0.3775 - val_binary_accuracy: 0.8611
Epoch 5/100
144/144 [==============================] - 0s 350us/step - loss: 0.2638 - binary_accuracy: 0.8889 - val_loss: 0.3458 - val_binary_accuracy: 0.8611
Epoch 6/100
144/144 [==============================] - 0s 343us/step - loss: 0.2593 - binary_a

## Testing

In [13]:
test = pd.read_csv('dataset/test_values.csv')

# Convert string values to integers
test["thal"] = test["thal"].astype('category')
test["thal"] = test["thal"].cat.codes

# Whiten test set
# test[cols] = (test[cols] - train_mean) / train_std

predictions = model.predict(test[cols].values)

output = pd.DataFrame({
    'patient_id': test['patient_id'],
    'heart_disease_present': predictions[:,0]
})

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(output)

output_dir = 'output'
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)
output_file = output_dir + '/' + experiment_name + '-' + 'out.csv'
output.to_csv(output_file, index=False)

,patient_id,heart_disease_present
0,olalu7,7.880373e-01
1,z9n6mx,5.230449e-03
2,5k4413,9.997579e-01
3,mrg7q5,4.287251e-01
4,uki4do,9.999565e-01
5,kev1sk,1.459868e-03
6,9n6let,2.792265e-03
7,jxmtyg,9.999993e-01
8,51s2ff,6.888431e-01
9,wi9mcs,1.961565e-01
